In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import glob
from scipy import stats
import matplotlib.pyplot as plt
import lightgbm as lgb

In [2]:
import os
os.chdir("/kaggle/input")

In [5]:
os.listdir()

['df_train (1)']

In [6]:
#Created datasets
df_train = pd.read_pickle('df_train (1)')

In [7]:
fet = df_train.columns[df_train.columns.str.contains('order_dow')]
fet2 = df_train.columns[df_train.columns.str.contains('order_hour')]

In [8]:
fet = fet2.append(fet)

In [9]:
for column in fet:
    df_train[column] = df_train[column].astype(np.uint8)

In [20]:
df_train[['aisle_id', 'department_id']] = df_train[['aisle_id', 'department_id']].astype('category')

In [21]:
X = df_train.drop(['order_id', 'product_id', 'y'], axis=1)
y = df_train['y'].values

In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state=30, shuffle = True, stratify = y)

In [ ]:
# #Feature Scaling
# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# X_train= sc_X.fit_transform(X_train)
# X_test= sc_X.fit_transform(X_test)

In [23]:
#light_gbm
dataset_train = lgb.Dataset(X_train, label=y_train, categorical_feature=['aisle_id', 'department_id'])

In [24]:
params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': {'binary_logloss'},
    'num_leaves': 96,
    'max_depth': 10,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.95,
    'bagging_freq': 5
}
ROUNDS = 100

In [25]:
model = lgb.train(params, dataset_train, ROUNDS)

/opt/conda/lib/python3.6/site-packages/lightgbm/basic.py:1186: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


In [60]:
print('light GBM predict')
preds = model.predict(X_test)

light GBM predict


In [61]:
preds[preds>=0.1] = 1
preds[preds<.1] = 0 

In [62]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print("Accuracy Score:", accuracy_score(y_test, preds))
print("Precision Score:", precision_score(y_test, preds))
print("Recall Score:", recall_score(y_test, preds))
print("f1 Score:", f1_score(y_test, preds))

Accuracy Score: 0.6152802923537966
Precision Score: 0.15459218558524773
Recall Score: 0.6565170703849232
f1 Score: 0.25025582691306797


In [ ]:
# param_grid_dt = {'max_depth' : np.arange(1,11),
#                 'min_samples_leaf': [2,3,4,5]}
# cv_optimize(classifier_dt, param_grid_dt, X_train, y_train)